In [1]:
import pickle
import pandas as pd

from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from typing import Callable
from src.config import Config
from src.dataset import Interactions
from src.metrics import PrecisionAtK, RecallAtK, FOneScoreAtK
from src.optimizer import OptunaMaximizer

In [2]:
with Config.PREPROCESSED_INTERACTIONS_PATH.open("rb") as file_object:
    interactions = Interactions(pickle.load(file_object))

In [3]:
interactions.train = interactions.train[["user_id", "item_id", "interest_score"]]
interactions.test = interactions.test[["user_id", "item_id", "interest_score"]]
interactions.valid = interactions.valid[["user_id", "item_id", "interest_score"]]

In [4]:
interactions.train

,user_id,item_id,interest_score
0,126706,14433,0.400
1,127290,140952,0.290
2,66991,198453,0.445
3,46791,83486,0.615
4,79313,188770,0.940
...,...,...,...
1532993,153908,98585,0.220
1532994,154008,251969,0.020
1532995,154892,298192,0.840
1532996,156948,38118,0.890


In [5]:
interactions.test

,user_id,item_id,interest_score
1517914,38753,135245,0.000
1517915,101642,319500,0.835
1517916,13548,251184,0.000
1517917,130425,193445,0.490
1517918,93986,80733,0.235
...,...,...,...
1530838,141930,219928,0.450
1530839,53358,42887,0.290
1530840,151170,284652,0.135
1530841,141293,273421,0.240


In [6]:
interactions.valid

,user_id,item_id,interest_score
1503047,22032,287219,0.275
1503048,84214,121609,1.000
1503049,28992,11482,0.110
1503050,23345,281921,1.000
1503051,49466,2880,0.010
...,...,...,...
1517909,138587,291806,0.000
1517910,158991,99669,0.815
1517911,77232,142149,0.020
1517912,17843,174535,0.060


In [7]:
csr_train = csr_matrix(
    (interactions.train["interest_score"], 
    (interactions.train["user_id"], interactions.train["item_id"]))
)
csr_train

<159613x321752 sparse matrix of type '<class 'numpy.float32'>'
	with 1505202 stored elements in Compressed Sparse Row format>

In [8]:
class Objective:
    def __init__(self, train: csr_matrix, valid: pd.DataFrame, metric: Callable):
        self.train = train
        self.valid = valid
        self.metric = metric

    def __call__(self, trial) -> float:
        search_space = {
            "factors": trial.suggest_int("factors", 4, 256),
            "regularization": trial.suggest_float("regularization", 1e-8, 0.1),
            "iterations": trial.suggest_int("iterations", 8, 64)
        }
        model = AlternatingLeastSquares(**search_space)
        model.fit(self.train)
        predictions = pd.DataFrame(self.valid["user_id"].drop_duplicates())
        predictions["item_id"] = predictions["user_id"].apply(
            lambda user_id: 
                model.recommend(
                    user_id, 
                    self.train[user_id],
                    N=Config.K,
                    filter_already_liked_items=True
                )[0]
        )
        predictions = predictions.explode("item_id")
        return self.valid["user_id"].drop_duplicates().apply(
            lambda user_id: 
                self.metric.calculate(
                    predictions[predictions["user_id"] == user_id]["item_id"],
                    self.valid[self.valid["user_id"] == user_id]["item_id"]
                )
        ).mean()

In [9]:
recall = RecallAtK(Config.K)
precision = PrecisionAtK(Config.K)
fscore = FOneScoreAtK(Config.K)

In [10]:
fscore_objective = Objective(csr_train, interactions.valid, fscore)
fscore_optimizer = OptunaMaximizer(fscore_objective, "ALS fscore maximize")
fscore_best_trial = fscore_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-02-25 15:52:34,369] A new study created in memory with name: ALS fscore maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

[I 2023-02-25 15:52:47,413] Trial 0 finished with value: 0.0024454396292115294 and parameters: {'factors': 67, 'regularization': 0.023544345955209517, 'iterations': 45}. Best is trial 0 with value: 0.0024454396292115294.


  0%|          | 0/9 [00:00<?, ?it/s]

...

[I 2023-02-25 15:58:44,523] Trial 20 finished with value: 0.00382468968055457 and parameters: {'factors': 219, 'regularization': 0.019483799841571087, 'iterations': 16}. Best is trial 12 with value: 0.003922383537541398.


  0%|          | 0/16 [00:00<?, ?it/s]

[I 2023-02-25 15:59:04,283] Trial 21 finished with value: 0.0037601472840087602 and parameters: {'factors': 236, 'regularization': 0.018075167146484737, 'iterations': 16}. Best is trial 12 with value: 0.003922383537541398.


  0%|          | 0/26 [00:00<?, ?it/s]

[I 2023-02-25 15:59:26,956] Trial 22 finished with value: 0.0036645160196982197 and parameters: {'factors': 256, 'regularization': 0.008097060188307408, 'iterations': 26}. Best is trial 12 with value: 0.003922383537541398.
EarlyStopping Exceeded: No new best scores on iters 10


In [11]:
precision_objective = Objective(csr_train, interactions.valid, precision)
precision_optimizer = OptunaMaximizer(precision_objective, "ALS precision maximize")
precision_best_trial = precision_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-02-25 15:59:27,048] A new study created in memory with name: ALS precision maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

[I 2023-02-25 15:59:52,850] Trial 0 finished with value: 0.0022864828513786202 and parameters: {'factors': 233, 'regularization': 0.051641405702684384, 'iterations': 46}. Best is trial 0 with value: 0.0022864828513786202.


  0%|          | 0/19 [00:00<?, ?it/s]

...

[I 2023-02-25 16:09:12,026] Trial 30 finished with value: 0.0023345182053991793 and parameters: {'factors': 235, 'regularization': 0.02380086566614388, 'iterations': 52}. Best is trial 22 with value: 0.002853300028821222.


  0%|          | 0/13 [00:00<?, ?it/s]

[I 2023-02-25 16:09:33,165] Trial 31 finished with value: 0.002766836391584215 and parameters: {'factors': 253, 'regularization': 0.0344070289994648, 'iterations': 13}. Best is trial 22 with value: 0.002853300028821222.


  0%|          | 0/12 [00:00<?, ?it/s]

[I 2023-02-25 16:09:53,435] Trial 32 finished with value: 0.002622730329522537 and parameters: {'factors': 254, 'regularization': 0.033322651661722144, 'iterations': 12}. Best is trial 22 with value: 0.002853300028821222.
EarlyStopping Exceeded: No new best scores on iters 10


In [12]:
recall_objective = Objective(csr_train, interactions.valid, recall)
recall_optimizer = OptunaMaximizer(recall_objective, "ALS recall maximize")
recall_best_trial = recall_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-02-25 16:09:53,509] A new study created in memory with name: ALS recall maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

[I 2023-02-25 16:10:08,886] Trial 0 finished with value: 0.007478991317496458 and parameters: {'factors': 89, 'regularization': 0.004597020171128827, 'iterations': 54}. Best is trial 0 with value: 0.007478991317496458.


  0%|          | 0/35 [00:00<?, ?it/s]

...

[I 2023-02-25 16:15:54,697] Trial 19 finished with value: 0.010545100321375664 and parameters: {'factors': 169, 'regularization': 0.06901379176484686, 'iterations': 25}. Best is trial 11 with value: 0.012485529142989036.


  0%|          | 0/26 [00:00<?, ?it/s]

[I 2023-02-25 16:16:16,214] Trial 20 finished with value: 0.011120612963238703 and parameters: {'factors': 227, 'regularization': 0.09865752509021025, 'iterations': 26}. Best is trial 11 with value: 0.012485529142989036.


  0%|          | 0/13 [00:00<?, ?it/s]

[I 2023-02-25 16:16:33,689] Trial 21 finished with value: 0.011550672544033224 and parameters: {'factors': 217, 'regularization': 0.08392270413682602, 'iterations': 13}. Best is trial 11 with value: 0.012485529142989036.
EarlyStopping Exceeded: No new best scores on iters 10


In [13]:
precision_model = AlternatingLeastSquares(**precision_best_trial.params)
precision_model.fit(csr_train)

  0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
recall_model = AlternatingLeastSquares(**recall_best_trial.params)
recall_model.fit(csr_train)

  0%|          | 0/14 [00:00<?, ?it/s]

In [15]:
fscore_model = AlternatingLeastSquares(**fscore_best_trial.params)
fscore_model.fit(csr_train)

  0%|          | 0/21 [00:00<?, ?it/s]

In [18]:
models = {
    "fscore_model": fscore_model,
    "recall_model": recall_model,
    "precision_model": precision_model
}

In [19]:
metrics = {
    "precision": precision,
    "recall": recall
}

In [21]:
for model_name, model in models.items():
    pred = pd.DataFrame(interactions.test["user_id"].drop_duplicates())
    print(f"Model '{model_name}':")
    pred["item_id"] = pred["user_id"].apply(
        lambda user_id: 
            model.recommend(
                user_id, 
                csr_train[user_id],
                N=Config.K,
                filter_already_liked_items=True
            )[0]
    )
    pred = pred.explode("item_id")
    for metric_name, metric in metrics.items():
        metric_val = interactions.test["user_id"].drop_duplicates().apply(
            lambda user_id: 
                metric.calculate(
                    pred[pred["user_id"] == user_id]["item_id"],
                    interactions.test[interactions.test["user_id"] == user_id]["item_id"]
                )
        ).mean()
        print(f"\tMetric {metric_name}@{Config.K}: {metric_val}")

Model 'fscore_model':
	Metric precision@10: 0.0016728427739544705
	Metric recall@10: 0.008561443482698114
Model 'recall_model':
	Metric precision@10: 0.001736368448914767
	Metric recall@10: 0.00846431684308868
Model 'precision_model':
	Metric precision@10: 0.0019481206987824247
	Metric recall@10: 0.010116979367905784
